In [42]:
import scipy as sp
import numpy as np
from sympy import Matrix
from numpy.linalg import eig,inv,solve,eigh
from scipy.sparse.linalg import eigsh
from scipy.constants import hbar,electron_mass 
π,ħ,me = np.pi,hbar,electron_mass
inveig = lambda Es,Evects: Evects@(np.diag(Es))@inv(Evects)

def WAVE_1D_MAT(n,L=1,CORRECT_UNITS=True):
    x,Δx = np.linspace(0, L, n,endpoint=False, retstep=True)
    freqs = np.fft.fftfreq(n, Δx)
    ks = 2*π*freqs
    Evects = np.array([np.exp(1j*k*x) for k in ks]).T
    Es = ks**2*(ħ**2) / (2 * me) if CORRECT_UNITS else ks**2
    MATRIX = inveig(Es,Evects)
    return MATRIX
def WAVE_2D_MAT(n,L=1,CORRECT_UNITS=True):
    p,Δp = np.linspace(0, L, n,endpoint=False, retstep=True)
    P = np.meshgrid(p, p)
    freqs = np.fft.fftfreq(n, Δp)
    #ks = 2*π*freqs
    #Evects = np.array([np.exp(1j*k*p) for k in ks]).T
    freqs2D = np.meshgrid(freqs, freqs)
    a = (2*π*freqs2D[0]).flatten()
    b = (2*π*freqs2D[1]).flatten()
    Evects = np.array([(np.exp(1j*ai*P[0])*np.exp(1j*bi*P[1])).flatten() for ai,bi in zip(a,b)]).T
    Es = (a**2+b**2).flatten()*(ħ**2) / (2 * me) if CORRECT_UNITS else (a**2+b**2).flatten()
    return inveig(Es,Evects)

def WAVE_3D_MAT(n,L=1,CORRECT_UNITS=True):
    p,Δp = np.linspace(0, L, n,endpoint=False, retstep=True)
    P = np.meshgrid(p, p, p)
    freqs = np.fft.fftfreq(n, Δp)
    freqs3D = np.meshgrid(freqs, freqs, freqs)
    a = (2*π*freqs3D[0]).flatten()
    b = (2*π*freqs3D[1]).flatten()
    c = (2*π*freqs3D[2]).flatten()
    Evects = np.array([(np.exp(1j*ai*P[0])*np.exp(1j*bi*P[1])*np.exp(1j*ci*P[2])).flatten() for ai,bi,ci in zip(a,b,c)]).T
    Es = (a**2+b**2+c**2).flatten()*(ħ**2) / (2 * me) if CORRECT_UNITS else (a**2+b**2+c**2).flatten()
    return inveig(Es,Evects)
def MAT_1D_to_2D(Mat, EIGEN=False):
    A = sp.sparse.csr_array(Mat)
    MAT2D = sp.sparse.kronsum(A, A)
    return eigsh(MAT2D) if EIGEN else MAT2D.todense()
    
def MAT_1D_to_3D(Mat, EIGEN=False):
    n = len(Mat)
    A = sp.sparse.csr_array(Mat)
    I = sp.sparse.eye_array(n)
    MAT3D = sp.sparse.kron(sp.sparse.kron(I, I),A) + sp.sparse.kron(sp.sparse.kron(I, A),I) + sp.sparse.kron(sp.sparse.kron(A, I),I)
    return eigsh(MAT3D) if EIGEN else MAT3D.todense()

def WAVE_nD(n,D=1,L=1,CORRECT_UNITS=True,EIGEN=False):
    x,Δx = np.linspace(0, L, n,endpoint=False, retstep=True)
    freqs = np.fft.fftfreq(n, Δx)
    ks = 2*π*freqs
    Evects = np.array([np.exp(1j*k*x) for k in ks]).T
    Es = ks**2*(ħ**2) / (2 * me) if CORRECT_UNITS else ks**2
    MATRIX = inveig(Es,Evects)
    if D == 1:
        return eig(MATRIX) if EIGEN else MATRIX
    elif D == 2:
        A = sp.sparse.csr_array(MATRIX)
        MAT2D = sp.sparse.kronsum(A, A)
        return eigsh(MAT2D) if EIGEN else MAT2D.todense()
    else:
        I = sp.sparse.eye_array(n)
        A = sp.sparse.csr_array(MATRIX)
        kronMATs = []
        for d1 in range(D):
            if d1 == 0:
                P = A
            else:
                P = I
            for d2 in range(1,D):
                if d1 == d2:
                    P = sp.sparse.kron(P, A)
                else:
                    P = sp.sparse.kron(P, I)
            kronMATs.append(P)
        MATnD = sum(kronMATs)
        return eigsh(MATnD) if EIGEN else MATnD.todense()
            






n1:True,n2:True,n3:True,n4:True,n5:True,n6:True,n7:True,n8:True,n9:True,n1:True,n2:True,n3:True,n4:True,n5:True,n6:True,n7:True,n8:True,n9:True,n10:True,n11:True,

In [88]:
#CHECK EQUIVALENCE 2D MATRIC: Let the mathematicians prove it to be true for all n and L, but this is good enough for me
#I have checked low number of n with mathematica and it is equivalent as far as I can see
for n in range(1,10):
    #l = np.random.rand(1)[0]*5
    NEW_METHOD = MAT_1D_to_2D(WAVE_1D_MAT(n,CORRECT_UNITS=False))
    OLD_METHOD = WAVE_2D_MAT(n,CORRECT_UNITS=False)
    print(f"n{n}:{np.array_equal(np.round(NEW_METHOD,8), np.round(OLD_METHOD,8))}", end=",")
#CHECK EQUIVALENCE 3D MATRIC: Let the mathematicians prove it to be true for all n and L, but this is good enough for me
#I have checked low number of n with mathematica and it is equivalent as far as I can see    
for n in range(1,12):
    #l = np.random.rand(1)[0]*5
    NEW_METHOD = MAT_1D_to_3D(WAVE_1D_MAT(n,CORRECT_UNITS=False))
    OLD_METHOD = WAVE_3D_MAT(n,CORRECT_UNITS=False)
    print(f"n{n}:{np.array_equal(np.round(NEW_METHOD,8), np.round(OLD_METHOD,8))}", end=",")
Matrix(WAVE_nD(25,D=1,CORRECT_UNITS=True,EIGEN=True)[1])



n1:True,n2:True,n3:True,n4:True,n5:True,n6:True,n7:True,n8:True,n9:True,n1:True,n2:True,n3:True,n4:True,n5:True,n6:True,n7:True,n8:True,n9:True,n10:True,n11:True,

Matrix([
[                           0.282842712474618,                             0.28284271247462, 0.00265858426111622 - 0.000691653202654177*I,  0.0179743276154774 + 0.00972931825532217*I,                          0.282020369648335,  0.0397711166980698 - 0.0196886800237145*I,  0.200000000000004 + 4.5233245654874e-17*I,                           0.282626066280651,  0.0207375151246632 - 0.00347559676634555*I,                           0.282737547027529,  0.0073139844870811 + 0.000210250163099785*I,                          0.281806275954155,  0.0343086588415245 + 0.0123819728350614*I,   0.27378635591805 + 0.00838874387437563*I,  -0.0461322653257284 + 0.0063145994278761*I, 0.282124450738771 + 1.04083408558608e-16*I, 0.00358066383513717 - 0.00148349363603219*I,   0.252359270662435 + 0.0154119955974362*I, -0.0424896474832151 + 0.00960495187313135*I, -0.189407121992341 - 0.0871035424042398*I,  -0.123988412493623 + 0.0668099282793237*I,    0.277213448199623 - 0.0017820449106273*I,   0.132